<a href="https://colab.research.google.com/github/PeterRutkowski/fer/blob/master/paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint

import numpy as np

In [14]:
loaded = np.load('drive/My Drive/project42/KDEF_F.npz', allow_pickle=True)
x_train, y_train = np.expand_dims(loaded['x_train'], axis=3), loaded['y_train']
x_val, y_val = np.expand_dims(loaded['x_val'], axis=3), loaded['y_val']
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(686, 100, 100, 1) (686, 7) (294, 100, 100, 1) (294, 7)


In [15]:
cnn = keras.Sequential(
  [   
    layers.Input(shape=(100,100,1,)),

    layers.Conv2D(filters=20, kernel_size=(5,5), strides=(1,1), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    layers.Conv2D(filters=40, kernel_size=(5,5), strides=(1,1), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    
    layers.Conv2D(filters=60, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

    layers.Conv2D(filters=30, kernel_size=(3,3), strides=(1,1), padding='same'),
    layers.BatchNormalization(),
    layers.Activation('relu'),
   
    layers.Dense(100),
    layers.BatchNormalization(),
    layers.Activation('relu'),
   
    layers.MaxPooling2D(pool_size=(12,12)),
   
    layers.Flatten(),
    layers.Dense(7, activation='softmax'),
  ]
)

In [16]:
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 100, 100, 20)      520       
_________________________________________________________________
batch_normalization_10 (Batc (None, 100, 100, 20)      80        
_________________________________________________________________
activation_10 (Activation)   (None, 100, 100, 20)      0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 50, 50, 20)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 50, 50, 40)        20040     
_________________________________________________________________
batch_normalization_11 (Batc (None, 50, 50, 40)        160       
_________________________________________________________________
activation_11 (Activation)   (None, 50, 50, 40)       

In [18]:
initial_learning_rate = 0.1
decay_rate = 0.01
decay_steps = 1.0
n_epochs = 175

learning_rate_fn = keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate, decay_steps, decay_rate)

cnn.compile(optimizer=keras.optimizers.SGD(
                  learning_rate=learning_rate_fn), loss='categorical_crossentropy', metrics=['accuracy'])

filepath = 'drive/My Drive/project42/cnn.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

cnn.fit(x=x_train,
    y=y_train,
    batch_size=980,
    epochs=n_epochs,
    verbose=1,
    callbacks=callbacks_list,
    validation_split=0.0,
    validation_data=(x_val, y_val),
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,)

Epoch 1/175
1/1 [==============================] - ETA: 0s - loss: 0.1318 - accuracy: 1.0000
Epoch 00001: val_accuracy improved from -inf to 0.14286, saving model to drive/My Drive/project42/cnn.h5
1/1 [==============================] - 1s 1s/step - loss: 0.1318 - accuracy: 1.0000 - val_loss: 2.4967 - val_accuracy: 0.1429
Epoch 2/175
1/1 [==============================] - ETA: 0s - loss: 0.4551 - accuracy: 0.9067
Epoch 00002: val_accuracy improved from 0.14286 to 0.17687, saving model to drive/My Drive/project42/cnn.h5
1/1 [==============================] - 0s 142ms/step - loss: 0.4551 - accuracy: 0.9067 - val_loss: 3.8403 - val_accuracy: 0.1769
Epoch 3/175
1/1 [==============================] - ETA: 0s - loss: 1.9845 - accuracy: 0.2843
Epoch 00003: val_accuracy did not improve from 0.17687
1/1 [==============================] - 0s 85ms/step - loss: 1.9845 - accuracy: 0.2843 - val_loss: 2.5461 - val_accuracy: 0.1463
Epoch 4/175
1/1 [==============================] - ETA: 0s - loss: 3.4

In [ ]:
from tensorflow.keras.models import load_model
cnn = load_model('drive/My Drive/project42/cnn.h5')

x = np.zeros((1,100,100,1))
cnn.predict(x)